In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from sklearn.model_selection import train_test_split
import dakpy
import cv2
import numpy as np
import os

In [ ]:
#preping the file paths so that each is in numerical order


a = os.listdir('/Users/gem/Desktop/Corrosion Data/Mono/Ground Truth')
b = os.listdir('/Users/gem/Desktop/Corrosion Data/Mono/Depth')
c = os.listdir('/Users/gem/Desktop/Corrosion Data/Non Mono/Ground Truth')
d = os.listdir('/Users/gem/Desktop/Corrosion Data/Non Mono/Depth')

a.sort()
b.sort()
c.sort()
d.sort()

a = sorted(a, key=len)
b = sorted(b, key=len)
c = sorted(c, key=len)
d = sorted(d, key=len)

In [ ]:
#run this cell to remove unwanted items in each list

b.remove('.DS_Store')
c.remove('.DS_Store')
d.remove('.DS_Store')

In [ ]:
#run this cell to append the full path to each item in each list


for i in range(len(a)):
    a[i] = '/Users/gem/Desktop/Corrosion Data/Mono/Ground Truth/' + a[i]
for i in range(len(b)):
    b[i] = '/Users/gem/Desktop/Corrosion Data/Mono/Depth/' + b[i]
for i in range(len(c)):
    c[i] = '/Users/gem/Desktop/Corrosion Data/Non Mono/Ground Truth/' + c[i]
for i in range(len(d)):
    d[i] = '/Users/gem/Desktop/Corrosion Data/Non Mono/Depth/' + d[i]

In [ ]:
#alternative files for debugging

a = os.listdir('/Users/gem/Desktop/Corrosion Data Copy/Non Mono/Ground Truth')
b = os.listdir('/Users/gem/Desktop/Corrosion Data Copy/Non Mono/Depth')


a.sort()
b.sort()

a = sorted(a, key=len)
b = sorted(b, key=len)

a.remove('.DS_Store')
b.remove('.DS_Store')

for i in range(len(a)):
    a[i] = '/Users/gem/Desktop/Corrosion Data Copy/Non Mono/Ground Truth/' + a[i]
for i in range(len(b)):
    b[i] = '/Users/gem/Desktop/Corrosion Data Copy/Non Mono/Depth/' + b[i]
    
a = a[0:10]
b = b[0:10]

In [ ]:
#using the dakpy module to get the training images for the mono dataset. Each training image has a raduis of 5
#and a center pixel to classify and is therefore 11X11. That paramter could be changed here in the get_train fuction though
data = []
for i, j in zip(a,b):
    data.append(dakpy.get_train(i, j, 5))

In [ ]:
#using the dakpy module to get the training images for the non mono dataset.
data1 = []
for i, j in zip(c,d):
    data1.append(dakpy.get_train(i, j, 5))

In [ ]:
#this cell makes a list 
t = []
for i in range(25):      
    t.append(data[i][1])

In [ ]:
t1 = []
for i in range(25):
    t1.append(data1[i][1])

In [ ]:
z1_train = np.concatenate(t, axis=0)

In [ ]:
z2_train = np.concatenate(t1, axis=0)

In [ ]:
h = []
for i in range(25): 
    h.append(data[i][0][:, 0])

In [ ]:
h1 = []
for i in range(25):
    h1.append(data1[i][0][:, 0])

In [ ]:
w1_train = np.concatenate(h, axis=None)

In [ ]:
w2_train = np.concatenate(h1, axis=None)

In [ ]:
z_train = np.concatenate((z1_train, z2_train), axis=0)

In [ ]:
w_train = np.concatenate((w1_train, w2_train), axis=None)

In [ ]:
z1_train.shape

In [ ]:
w1_train.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(z1_train, w1_train, test_size = .2)

In [ ]:
#Give the negative labels 0 rather than -1

for i in range(len(y_train)):
    if y_train[i] < 0:
        y_train[i] = 0
for i in range(len(y_test)):
    if y_test[i] < 0:
        y_test[i] = 0

In [ ]:
#The CNN


batch_size = 128
num_classes = 2
epochs = 10

# input image dimensions
img_rows, img_cols = 11, 11

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

#parse the data type to 32 bit floats and normalize. The input images have been read in as 16 bit integers from the
#dakpy module
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 65535
x_test /= 65535

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices for use in categorical crossentropy
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
p_1 = model.predict(dakpy.get_full('/Users/gem/Desktop/Corrosion Data/Non Mono/Depth/Depth_non_1.png', 5).reshape(160000, 11, 11, 1))

In [ ]:
p_1 = np.delete(p_1, 1, 1)

In [ ]:
p_1 = p_1.reshape(400, 400)

In [ ]:
np.unique(p_1)

In [ ]:
for i in range(400):
    for j in range(400):
        if p_1[i, j] > .5:
            p_1[i, j] = 255
        else:
            p_1[i, j] = 0